## Lecture CSV

In [4]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.metrics import classification_report, confusion_matrix
import torch

In [5]:
# data_train = pd.read_csv("../datas/train_clean.csv")
data_train = pd.read_csv("/content/train_clean.csv")

data_train.head()

,keyword,location,text,labels
0,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


---

## distilbert-base-uncased

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased-finetuned-sst-2-english")

## jy46604790/Fake-News-Bert-Detect

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("jy46604790/Fake-News-Bert-Detect")
model = AutoModelForSequenceClassification.from_pretrained("jy46604790/Fake-News-Bert-Detect")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

## hamzab/roberta-fake-news-classification

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("hamzab/roberta-fake-news-classification")
model = AutoModelForSequenceClassification.from_pretrained("hamzab/roberta-fake-news-classification")

tokenizer_config.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

## cardiffnlp/twitter-roberta-base

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Charger le tokenizer
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base")

# Charger le modèle avec une tête de classification binaire
model = AutoModelForSequenceClassification.from_pretrained(
    "cardiffnlp/twitter-roberta-base",
    num_labels=2
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [7]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding=True)

Rempace les NaN pour pouvoir l'utiliser

In [ ]:
data_train["keyword"] = data_train["keyword"].fillna("")

In [ ]:
data_train["location"] = data_train["location"].fillna("")

Concatène toutes les colonnes

In [ ]:
data_train["all_concat"] = data_train["text"] + data_train["keyword"] + data_train["location"]

Preprocess text

In [8]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [9]:
data_train["text"] = data_train["text"].apply(preprocess)

In [10]:
data_train.head()

,keyword,location,text,labels
0,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


Split train / val

In [11]:
from datasets import Dataset

# Conversion pandas -> Hugging Face Dataset
hf_dataset = Dataset.from_pandas(data_train)

# Split 80% train, 20% test (ou val)
split_dataset = hf_dataset.train_test_split(test_size=0.2)

# Accès aux sous-datasets
train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]

# Appliquer la fonction de tokenization avec batching
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# Après tokenization, préciser les colonnes à utiliser et convertir en PyTorch tensors
tokenized_train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/6090 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1523 [00:00<?, ? examples/s]

In [12]:
from transformers import DataCollatorWithPadding

# Auto Padding : toutes les séquences sont de la même longueur
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
# On vérifie la taille des séquences
samples = tokenized_train_dataset[:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

[64, 64, 64, 64, 64, 64, 64, 64]

---

## Fine Tuning

In [14]:
from transformers import TrainingArguments

# paramètres d'entrainement par défaut
# training_args = TrainingArguments("test-trainer", report_to="none")

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

In [15]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

In [16]:
from sklearn.utils.class_weight import compute_class_weight
from transformers import Trainer

labels = data_train["labels"].values
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(labels), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float)


class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights.to(device))
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [17]:
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-2387202010.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(


---

## Train

In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.364212,0.846356,0.827941,0.836553,0.819505
2,0.427600,0.440810,0.822718,0.810659,0.782138,0.841339
3,0.296400,0.450075,0.837163,0.819767,0.818578,0.820961
4,0.215800,0.540510,0.842416,0.823270,0.833085,0.813683


TrainOutput(global_step=1524, training_loss=0.31133312096433063, metrics={'train_runtime': 540.5917, 'train_samples_per_second': 45.062, 'train_steps_per_second': 2.819, 'total_flos': 874294398077400.0, 'train_loss': 0.31133312096433063, 'epoch': 4.0})

---

## Predictions

In [19]:
predictions = trainer.predict(tokenized_val_dataset)

In [20]:
preds = np.argmax(predictions.predictions, axis=-1)

---

## Classification report

In [21]:
print(classification_report(tokenized_val_dataset["labels"], preds))

              precision    recall  f1-score   support

           0       0.85      0.87      0.86       836
           1       0.84      0.82      0.83       687

    accuracy                           0.85      1523
   macro avg       0.85      0.84      0.84      1523
weighted avg       0.85      0.85      0.85      1523



---

## Confusion Matrix

In [22]:
# produce the confusion matrix for your predictions, what comments can you make ?
mat = confusion_matrix(tokenized_val_dataset["labels"], preds)

labels = data_train["labels"].unique()
df_mat = pd.DataFrame(mat, index=labels, columns=labels)

fig = px.imshow(df_mat, text_auto=True, color_continuous_scale=px.colors.sequential.Aggrnyl,
    labels=dict(x="Prédiction", y="Réalité", color="Nombre"))
fig.update_coloraxes(showscale=False)
fig.show()